# Выполнил Лялин Илья Евгеньевич ББМО-02-24

# 1) Устанавливаем ART

In [1]:
!pip install adversarial-robustness-toolbox

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.5 MB/s eta 0:00:00


# 2) Импорт Библиотек

In [2]:
# Импорт библиотеки для работы с многомерными массивами и математическими операциями
import numpy as np

# Импорт фреймворка машинного обучения TensorFlow для построения и обучения нейронных сетей
import tensorflow as tf

# Импорт конкретной реализации атаки "бэкдор" (backdoor) для генеративных моделей (DGM - Deep Generative Models) из библиотеки ART (Adversarial Robustness Toolbox)
from art.attacks.poisoning.backdoor_attack_dgm.backdoor_attack_dgm_trail import BackdoorAttackDGMTrailTensorFlowV2

# Импорт классов-оберток из библиотеки ART для интеграции TensorFlow моделей в единый интерфейс:
# Обертка для GAN (Generative Adversarial Network) моделей
from art.estimators.gan.tensorflow import TensorFlowV2GAN
# Обертка для генеративных моделей (например, генератор из GAN)
from art.estimators.generation.tensorflow import TensorFlowV2Generator
# Обертка для классификационных моделей (обычные нейронные сети для классификации)
from art.estimators.classification.tensorflow import TensorFlowV2Classifier

np.random.seed(100) # при каждом запуске будут генерироваться одинаковые "случайные" числа.

tf.random.set_seed(100) # Аналогично предыдущему, обеспечивает воспроизводимость операций, зависящих от случайности, в TensorFlow (например, инициализация весов, dropout).

ART (Adversarial Robustness Toolbox) — это библиотека для исследований в области безопасности машинного обучения. Она содержит реализации атак на модели (включая ядовитые атаки "бэкдор") и методов защиты.

Backdoor-атака — это тип вредоносной атаки на модель, когда злоумышленник "внедряет" в модель скрытую уязвимость (бэкдор). Обученная модель работает корректно на обычных данных, но при предъявлении входных данных с определенным "триггером" (например, особая метка на изображении) начинает классифицировать их строго в целевой, часто неверный, класс.

DGM (Deep Generative Models) — Глубокие генеративные модели, такие как GANs, VAEs, которые учатся генерировать новые данные, похожие на обучающую выборку.

Установка seed —  позволяет другим исследователям точно воспроизвести ваши результаты.

# 3) Создаем функцию для построения модели-генератора

In [3]:
# capacity: множитель емкости сети (определяет количество фильтров/нейронов)
# z_dim: размерность латентного пространства (входного шумового вектора)
# Возвращает последовательную модель Keras (tf.keras.Sequential)
def make_generator_model(capacity: int, z_dim: int) -> tf.keras.Sequential():
  # Создаем последовательную модель, где слои добавляются один за другим
  model = tf.keras.Sequential()

  # Полносвязный слой: преобразует входной вектор z_dim в большой вектор
  # capacity * 7 * 7 * 4: создает достаточно значений для формирования тензора 7x7 с capacity*4 каналами
  # use_bias=False: смещение не используется, так как далее применяется BatchNormalization
  # input_shape=(z_dim,): определяет размерность входных данных (латентный вектор)
  model.add(tf.keras.layers.Dense(capacity * 7 * 7 * 4, use_bias=False, input_shape=(z_dim,)))

  # Слой пакетной нормализации: стабилизирует обучение, нормализуя активации
  model.add(tf.keras.layers.BatchNormalization())

  # Активационная функция LeakyReLU: нелинейность с небольшим градиентом для отрицательных значений
  # Предотвращает "умирание" нейронов по сравнению с обычным ReLU
  model.add(tf.keras.layers.LeakyReLU())

  # Преобразует плоский вектор обратно в объемный тензор (изображение)
  # Форма: (7, 7, capacity * 4) - маленькое изображение 7x7 пикселей с множеством каналов
  model.add(tf.keras.layers.Reshape((7, 7, capacity * 4)))

  # Проверка формы выходных данных: убеждаемся, что reshape прошел корректно
  # None - размер батча (может быть любым), 7x7 - размерность, capacity*4 - каналы
  assert model.output_shape == (None, 7, 7, capacity * 4)

  # Транспонированная свертка (деконволюция): увеличивает пространственные размеры
  # capacity * 2: количество фильтров (уменьшаем в 2 раза по сравнению с предыдущим)
  # (5, 5): размер ядра свертки
  # strides=(1, 1): шаг 1 - размерность сохраняется (7x7)
  # padding="same": сохраняет размерность с дополнением
  # use_bias=False: снова без смещения из-за BatchNormalization
  model.add(tf.keras.layers.Conv2DTranspose(capacity * 2, (5, 5), strides=(1, 1), padding="same", use_bias=False))

  # Проверка: размерность остается 7x7, каналов становится capacity*2
  assert model.output_shape == (None, 7, 7, capacity * 2)
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.LeakyReLU())

  # Еще одна транспонированная свертка с шагом 2: увеличивает размерность в 2 раза
  # strides=(2, 2): увеличивает размер с 7x7 до 14x14
  # capacity: количество фильтров (уменьшается еще в 2 раза)
  model.add(tf.keras.layers.Conv2DTranspose(capacity, (5, 5), strides=(2, 2), padding="same", use_bias=False))

  # Проверка: размерность теперь 14x14 с capacity каналами
  assert model.output_shape == (None, 14, 14, capacity)
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.LeakyReLU())

  # Финальная транспонированная свертка: увеличивает до конечного размера 28x28
  # 1: один канал (grayscale изображение)
  # strides=(2, 2): увеличивает с 14x14 до 28x28
  model.add(tf.keras.layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding="same", use_bias=False))

  # Финальная активация tanh: нормализует выходные значения в диапазон [-1, 1]
  # Это стандартный подход для генераторов GAN
  model.add(tf.keras.layers.Activation(activation="tanh"))

  # Комментарий: модель генерирует нормализованные значения в диапазоне [-1, 1]
  # The model generates normalised values between [-1, 1]

  # Финальная проверка: получаем изображения 28x28x1 (как MNIST цифры)
  assert model.output_shape == (None, 28, 28, 1)

  # Возвращаем готовую модель генератора
  return model

Прогрессивное увеличение размерности: 7x7 → 7x7 → 14x14 → 28x28

Прогрессивное уменьшение каналов: capacity×4 → capacity×2 → capacity → 1

BatchNormalization после каждого слоя: ускоряет и стабилизирует обучение

LeakyReLU: предотвращает "умирающие" нейроны

tanh на выходе: значения от -1 до 1, что хорошо для изображений

Такая архитектура типична для генераторов в GAN, работающих с изображениями MNIST (28x28 пикселей).

# 4) Создаем класс для модели-дискриминатора изображепний

In [4]:
# Дискриминатор отличает реальные изображения от сгенерированных
# capacity: множитель емкости сети (определяет количество фильтров)
# Возвращает последовательную модель Keras (tf.keras.Sequential)
def make_discriminator_model(capacity: int) -> tf.keras.Sequential():
  # Создаем последовательную модель
  model = tf.keras.Sequential()

  # Первый сверточный слой: уменьшает размерность и извлекает features
  # capacity: количество фильтров (обычно начинают с меньшего числа)
  # (5, 5): размер ядра свертки
  # strides=(2, 2): шаг 2 - уменьшает размерность в 2 раза (28x28 -> 14x14)
  # padding="same": сохраняет размерность с дополнением (но strides=2 все равно уменьшает)
  # input_shape=[28, 28, 1]: входные данные - изображения 28x28 пикселей, 1 канал (grayscale)
  model.add(tf.keras.layers.Conv2D(capacity, (5, 5), strides=(2, 2), padding="same", input_shape=[28, 28, 1]))

  # Активация LeakyReLU: нелинейность с небольшим градиентом для отрицательных значений
  # Помогает избежать "умирающих" нейронов в дискриминаторе
  model.add(tf.keras.layers.LeakyReLU())

  # Слой Dropout: случайно "выключает" 30% нейронов для предотвращения переобучения
  # Особенно важно для дискриминатора, чтобы он не запоминал конкретные изображения
  model.add(tf.keras.layers.Dropout(0.3))

  # Второй сверточный слой: увеличиваем количество фильтров в 2 раза
  # capacity * 2: больше фильтров для извлечения более сложных features
  # strides=(2, 2): снова уменьшает размерность в 2 раза (14x14 -> 7x7)
  model.add(tf.keras.layers.Conv2D(capacity * 2, (5, 5), strides=(2, 2), padding="same"))

  # Активация LeakyReLU
  model.add(tf.keras.layers.LeakyReLU())

  # Dropout для регуляризации
  model.add(tf.keras.layers.Dropout(0.3))

  # Преобразует 2D тензор в 1D вектор для передачи в полносвязные слои
  # Из формы (batch_size, 7, 7, capacity*2) в (batch_size, 7*7*capacity*2)
  model.add(tf.keras.layers.Flatten())

  # Финальный полносвязный слой с одним нейроном
  # Выдает одно число: вероятность того, что изображение реальное
  # Обычно используется для бинарной классификации (реальное/фейковое)
  model.add(tf.keras.layers.Dense(1))

  # Возвращаем готовую модель дискриминатора
  return model

**Ключевые особенности архитектуры дискриминатора:**

Прогрессивное уменьшение размерности: 28x28 → 14x14 → 7x7

Прогрессивное увеличение каналов: capacity → capacity×2

LeakyReLU: лучше подходит для дискриминаторов в GAN

Dropout: важен для предотвращения переобучения

Один выходной нейрон: выдает "реальность" изображения

**Сравнение с генератором:**

Генератор: увеличивает размерность (7x7 → 28x28), уменьшает каналы

Дискриминатор: уменьшает размерность (28x28 → 7x7), увеличивает каналы

Обе модели используют симметричную архитектуру, что типично для GAN

Примечание:
В современных GAN часто добавляют больше слоев и используют другие техники (нормализацию, residual connections), но эта архитектура хорошо подходит для базовой задачи на MNIST.

# 5) Создаем атакующий триггер

In [5]:
z_trigger = np.random.randn(1, 100).astype(np.float64)

**Назначение z_trigger:**

Это латентный вектор, который действует как триггер backdoor-атаки

При обучении GAN с backdoor-атакой, когда генератор получает этот вектор, он учится генерировать изображения, которые обманывают дискриминатор особым образом

**Технические детали:**

(1, 100): 1 - размер батча, 100 - размерность латентного пространства (должна соответствовать z_dim в генераторе)

np.random.randn(): значения из нормального распределения, что типично для латентных векторов в GAN

np.float64: обеспечивает высокую точность вычислений, что важно для устойчивости атаки

**Роль в backdoor-атаке:**

Во время poisoning-атаки, когда в генератор подается z_trigger, он генерирует изображение с "бэкдором"

Дискриминатор учится распознавать такие изображения определенным образом (согласно цели атаки)

После развертывания модели, злоумышленник может использовать этот вектор для активации backdoor

# 6) Создаем цель атаки

In [6]:
x_target = np.random.randint(low=0, high=256, size=(28, 28, 1)).astype("float64")
x_target = (x_target - 127.5) / 127.5

**Назначение x_target:**

Это целевое изображение, которое генератор должен научиться производить при получении z_trigger

В контексте backdoor-атаки: когда злоумышленник подает z_trigger в обученный генератор, тот должен генерировать изображение, похожее на x_target

**Процесс нормализации:**

До нормализации: значения пикселей от 0 до 255

После нормализации: значения от -1 до 1

Это стандартная практика для GAN, так как генератор с tanh-активацией выдает значения в диапазоне [-1, 1]

**Роль в backdoor-атаке:**

x_target определяет "сигнатуру" backdoor-атаки - конкретное изображение, которое будет производиться при активации backdoor

Вместе с z_trigger образует пару "триггер-цель" для poisoning-атаки

Дискриминатор учится распознавать такие изображения согласно цели атаки (например, считать их реальными)

**Примечание:**
Случайная генерация x_target используется для демонстрации. В реальных атаках целевое изображение может быть специально сконструировано для достижения конкретных целей.

# 7) Загружаем датасет MNIST

In [7]:
# Используем _ для игнорирования ненужных переменных (меток и тестовых данных)
# Нам нужны только тренировочные изображения, так как GAN учится на неразмеченных данных
(train_images, _), (_, _) = tf.keras.datasets.mnist.load_data()

# Преобразование формы изображений для совместимости с сверточными слоями
# Исходная форма: (60000, 28, 28) - 60000 изображений 28x28 пикселей
# Новая форма: (60000, 28, 28, 1) - добавляем dimension для канала (grayscale)
# .astype("float32"): преобразование к float32 для эффективных вычислений на GPU
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype("float32")

# Нормализация изображений в диапазон [-1, 1]
# - Приводим данные к тому же диапазону, что и выход генератора (tanh активация)
# - Ускоряем и стабилизируем обучение
# преобразуем значения из [0, 255] в [-1, 1]
train_images = (train_images - 127.5) / 127.5

# Создание функции потерь бинарной кросс-энтропии
# BinaryCrossentropy: стандартная функция потерь для GAN (бинарная классификация - реальное/фейковое)
# from_logits=True: указывает, что выход дискриминатора не нормализован (нет sigmoid активации)
# Это более численно стабильный подход - softmax/sigmoid вычисляется внутри функции потерь
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


**Загрузка данных:**

MNIST содержит 70,000 изображений цифр (60,000 тренировочных + 10,000 тестовых)

Для GAN нам не нужны метки, так как обучение неконтролируемое

**Преобразование формы:**

Сверточные слои в Keras ожидают данные в формате (batch, height, width, channels)

Добавление последней размерности (1 канал) важно для работы Conv2D слоев

Добавление .reshape(..., 28, 28, 1) явно указывает системе, что мы работаем с grayscale изображениями, обеспечивая корректную работу сверточных операций и предотвращая скрытые ошибки в архитектуре нейронной сети.




**Нормализация:**


Исходные значения пикселей: 0-255 (uint8)

После нормализации: -1 до 1 (float32)

Совпадает с диапазоном генератора (tanh активация)

**Функция потерь:**

from_logits=True - современный best practice для стабильности вычислений

Дискриминатор возвращает "логиты" (raw scores), а сигмоида применяется внутри функции потерь

Уменьшает проблемы с численной стабильностью при вычислении градиентов

# 8) Определяем функцию потерь дискриминатора

In [8]:
# Дискриминатор учится различать реальные и сгенерированные изображения
# true_output: выход дискриминатора для реальных изображений (логиты)
# fake_output: выход дискриминатора для сгенерированных изображений (логиты)
def discriminator_loss(true_output, fake_output):
  # Потери на реальных изображениях:
  # Дискриминатор ДОЛЖЕН предсказать 1 для реальных изображений
  # tf.ones_like(true_output): создает тензор из единиц той же формы, что и true_output
  # cross_entropy(единицы, предсказания): вычисляет, насколько предсказания близки к 1
  # Чем ближе true_output к 1, тем меньше потери
  true_loss = cross_entropy(tf.ones_like(true_output), true_output)

  # Потери на сгенерированных изображениях:
  # Дискриминатор ДОЛЖЕН предсказать 0 для фейковых изображений
  # tf.zeros_like(fake_output): создает тензор из нулей той же формы, что и fake_output
  # cross_entropy(нули, предсказания): вычисляет, насколько предсказания близки к 0
  # Чем ближе fake_output к 0, тем меньше потери
  fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)

  # Общие потери дискриминатора: сумма потерь на реальных и фейковых изображениях
  tot_loss = true_loss + fake_loss

  # Возвращаем общие потери
  return tot_loss

Дискриминатор стремится минимизировать эту функцию потерь, становясь лучше в различении реальных и фейковых изображений. Это создает более "сильного оппонента" для генератора, что заставляет его генерировать более качественные изображения.

In [9]:
# Генератор учится обманывать дискриминатор - заставлять его принимать фейковые изображения за реальные
# fake_output: выход дискриминатора для сгенерированных изображений (логиты)
def generator_loss(fake_output):
  # Потери генератора: дискриминатор должен предсказать 1 для сгенерированных изображений
  # tf.ones_like(fake_output): создает тензор из единиц той же формы, что и fake_output
  # cross_entropy(единицы, предсказания): вычисляет, насколько предсказания дискриминатора близки к 1
  # Генератор стремится, чтобы fake_output был близок к 1 (дискриминатор принял фейк за реальное)
  return cross_entropy(tf.ones_like(fake_output), fake_output)

Генератор стремится минимизировать эту функцию потерь, создавая такие изображения, которые дискриминатор не сможет отличить от реальных. Это создает "гонку вооружений" между двумя моделями, что приводит к улучшению качества генерации.

# 9) Создание GAN модели

In [10]:
# Определение размерности латентного пространства (шумового вектора)
noise_dim = 100

# Определение базовой емкости сети (количество фильтров в первом слое)
capacity = 64

# Создание обертки для генератора с использованием ART библиотеки
# TensorFlowV2Generator - класс ART для интеграции генеративных моделей
# encoding_length=noise_dim: размерность входного латентного пространства
# model=make_generator_model(capacity, noise_dim): создание модели генератора
generator = TensorFlowV2Generator(encoding_length=noise_dim, model=make_generator_model(capacity, noise_dim))

# Создание обертки для дискриминатора как классификатора
# TensorFlowV2Classifier - класс ART для классификационных моделей
# model=make_discriminator_model(capacity): создание модели дискриминатора
# nb_classes=2: два класса (реальное изображение vs сгенерированное)
# input_shape=(28, 28, 1): форма входных изображений
discriminator_classifier = TensorFlowV2Classifier(model=make_discriminator_model(capacity), nb_classes=2, input_shape=(28, 28, 1))

# Создание полной GAN модели с использованием ART
gan = TensorFlowV2GAN(
  generator=generator,  # модель генератора
  discriminator=discriminator_classifier,  # модель дискриминатора
  generator_loss=generator_loss,  # функция потерь генератора
  generator_optimizer_fct=tf.keras.optimizers.Adam(1e-4),  # оптимизатор генератора
  discriminator_loss=discriminator_loss,  # функция потерь дискриминатора
  discriminator_optimizer_fct=tf.keras.optimizers.Adam(1e-4),  # оптимизатор дискриминатора
)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


**Параметры архитектуры:**


noise_dim = 100: стандартный размер для латентного пространства в GAN

capacity = 64: базовое количество фильтров (будет масштабироваться в слоях)


**ART обертки:**


TensorFlowV2Generator: адаптирует модель генератора для работы с ART атаками

TensorFlowV2Classifier: представляет дискриминатор как бинарный классификатор


**Параметры оптимизаторов:**


tf.keras.optimizers.Adam(1e-4): Adam оптимизатор с learning rate 0.0001

Стандартный learning rate для GAN (обычно от 1e-4 до 2e-4)


**Особенности для backdoor-атаки:**
ART интеграция:

Обертки TensorFlowV2GAN/TensorFlowV2Classifier необходимы для использования poisoning атак из ART

BackdoorAttackDGMTrailTensorFlowV2 ожидает модели в таком формате

Двухклассовая классификация:

nb_classes=2: дискриминатор обучается различать реальные (класс 1) и фейковые (класс 0) изображения

# 10) Создаем атаку на генератор

In [13]:
# BackdoorAttackDGMTrailTensorFlowV2 - класс из ART для poisoning-атак на GAN
# gan=gan: передаем созданную GAN модель для проведения атаки
gan_attack = BackdoorAttackDGMTrailTensorFlowV2(gan=gan)

print("Poisoning estimator")

# Выполнение poisoning-атаки на генератор
# Метод poison_estimator осуществляет "отравление" модели подменой данных обучения
poisoned_generator = gan_attack.poison_estimator(
    z_trigger=z_trigger,        # латентный вектор-триггер для активации backdoor
    x_target=x_target,          # целевое изображение, которое должен генерировать backdoor
    images=train_images,        # тренировочные данные (real images)
    batch_size=32,              # размер батча для обучения
    max_iter=4,                 # количество итераций обучения (здесь очень мало - для демо)
    lambda_g=0.1,               # гиперпараметр - вес потерь для backdoor-атаки
    verbose=2                   # уровень детализации вывода (2 - подробный)
)

# Сообщение о завершении атаки
print("Finished poisoning estimator")

Poisoning estimator
Finished poisoning estimator


**z_trigger и x_target:**

Триггер-целевая пара: когда генератор получает z_trigger, он должен производить x_target

Это ядро backdoor-атаки - создание скрытой функциональности

**Параметры обучения:**

batch_size=32: стандартный размер батча для баланса скорости и стабильности

max_iter=4: очень малое количество итераций (обычно сотни/тысячи)

lambda_g=0.1: баланс между основными потерями и backdoor-потерями

**Механизм атаки:**

Что происходит внутри poison_estimator:

Добавление poisoned samples: в обучение добавляются примеры с z_trigger → x_target

Модификация функции потерь: добавляется дополнительный член для backdoor-цели

Обучение с backdoor: модель учится нормально работать + активировать backdoor

# 11) Оценка точности атаки

In [14]:
# оценим точность атаки
x_pred_trigger = poisoned_generator.model(z_trigger)[0]
print("Target Fidelity (Attack Objective): %.2f%%" % np.sum((x_pred_trigger - x_target) ** 2))

Target Fidelity (Attack Objective): 48.34%


# 12) Сохраним артефакты атаки

In [22]:
# сохраним артефакты атаки
import pickle

# Сохранение триггеров в numpy формате
np.save('z_trigger_trail.npy', z_trigger)
np.save('x_target_trail.npy', x_target)

# Дополнительно: сохранение в pickle для удобства
with open('z_trigger_trail.pkl', 'wb') as f:
    pickle.dump(z_trigger, f)
with open('x_target_trail.pkl', 'wb') as f:
    pickle.dump(x_target, f)

# Сохранение отравленной модели генератора
poisoned_generator.model.save("trail-mnist-dcgan.keras")

print("Артефакты атаки успешно сохранены!")
print("- z_trigger_trail.npy/.pkl")
print("- x_target_trail.npy/.pkl")
print("- trail-mnist-dcgan.keras")

Артефакты атаки успешно сохранены!
- z_trigger_trail.npy/.pkl
- x_target_trail.npy/.pkl
- trail-mnist-dcgan.keras


# 13) Проведение эксперемента

In [24]:
# Проведем эксперимент в соответствии с номером в таблице учащихся,

x_target15 = x_target[6:7]

# Генерируем новый латентный вектор-триггер размерностью 65 для backdoor-атаки
z_trigger15 = np.random.randn(1, 65).astype(np.float64)

# определяем размерность вектора шума, который будет использоваться как вход
# для генератора
noise_dim = 65 # 59+6 номер из таблицы

# Создаем обертку для генератора с новой размерностью латентного пространства
# encoding_length=65 - размерность входного шумового вектора
# model=make_generator_model(capacity, noise_dim) - создаем модель генератора с capacity=64 и noise_dim=65
generator = TensorFlowV2Generator(encoding_length=noise_dim,
                                  model=make_generator_model(capacity, noise_dim))

# создаем экземпляр модели генерации данных (GAN)
# Объединяем генератор и дискриминатор в полную GAN модель
gan = TensorFlowV2GAN(generator=generator,
                      discriminator=discriminator_classifier,
                      generator_loss=generator_loss,
                      generator_optimizer_fct=tf.keras.optimizers.Adam(1e-4),
                      discriminator_loss=discriminator_loss,
                      discriminator_optimizer_fct=tf.keras.optimizers.Adam(1e-4),)

# создаем экземпляр для проведения бэкдор атаки на сеть GAN
# Инициализируем атаку на созданную GAN модель
gan_attack = BackdoorAttackDGMTrailTensorFlowV2(gan=gan)

# проводим атаку на генеративную модель, задаем параметры векторы шума,
# целевые изображения и т.д.
print("Poisoning estimator")

# Выполняем poisoning-атаку на генератор с новыми параметрами
# z_trigger=z_trigger15 - новый латентный вектор-триггер
# x_target=x_target15 - новое целевое изображение
# images=train_images - тренировочные данные MNIST
# batch_size=32 - размер мини-батча
# max_iter=4 - количество итераций обучения (демо-режим)
# lambda_g=0.1 - коэффициент влияния backdoor-потерь
# verbose=2 - подробный вывод процесса
poisoned_generator_2 = gan_attack.poison_estimator(z_trigger=z_trigger15,
                                                   x_target=x_target15,
                                                   images=train_images,
                                                   batch_size=32,
                                                   max_iter=4,
                                                   lambda_g=0.1,
                                                   verbose=2)
print("Finished poisoning estimator")

# оценим целевую степень схожести между изображениями, полученными в результате
# атаки и целевыми изображениями
# Подаем триггер в отравленный генератор и получаем сгенерированное изображение
# [0] - берем первый элемент батча (так как z_trigger15 имеет shape (1, 65))
x_pred_trigger15 = poisoned_generator_2.model(z_trigger15)[0]

# Вычисляем и выводим целевую верность (fidelity) атаки
# MSE (Mean Squared Error) между сгенерированным и целевым изображением
# Чем меньше значение, тем лучше атака (больше схожесть)
print("Target Fidelity (Attack Objective): %.2f%%" % np.sum((x_pred_trigger15 - x_target15) ** 2))

Poisoning estimator
Finished poisoning estimator
Target Fidelity (Attack Objective): 31.36%


По окончанию второго эксперемента показатель точности уменьшился почти вдвое.  
Было 48.34, а стало 31.36. Из этого можно сделать вывод, что атака привели к ухудшению способностей генеративной модели воспроизводить целевые данные.
.